In [111]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple
from numba import njit

In [312]:
arr = np.array
costs = []
alpha = 0.01

class LinearReg(object):
    def __init__(self, optim:str = "grad"):
        super(LinearReg, self).__init__()
        if optim == "grad":
            pass
            #self.fit = self.fit_grad
        elif optim == "ols":
            pass
            #self.fit = self.fit_ols
        else:
            raise Exception("Argument fit should be either ols or grad")

@njit
def forward(X:arr, W: arr, b:arr) -> arr:
    return np.dot(X,W) + b

@njit
def loss(y:arr, y_hat:arr) -> np.float:
    return np.mean((y-y_hat)**2)

@njit
def get_grads(X: arr, y: arr, y_hat: arr) -> Tuple[np.float, arr]:
    b_hat_d = -2 * (y-y_hat).mean()
    W_hat_d = -2 * np.dot((y-y_hat), X)  / y.shape[0]    
    return b_hat_d, W_hat_d

@njit
def update_params(W_hat:arr, b_hat:np.float, W_hat_d: arr, b_hat_d: np.float, alpha: np.float) -> Tuple[arr]:
    b_hat -= b_hat_d * alpha
    W_hat -= W_hat_d * alpha
    return b_hat, W_hat

X = np.random.normal(size=(1000, 10))
W = np.array([0.1, 0.2, -0.1, 0.2, 0.1, 0.1, 0.2, -0.1, 0.2, 0.1])
b = 0.5
y = np.dot(X,W) + b


@njit
def fit_grad(X, y, iterations: int = 100000):
    costs = []
    W_hat = np.random.randn(X.shape[1])
    b_hat = np.random.rand()
    for iteration in range(iterations):
        y_hat = forward(X, W_hat, b_hat)
        cost = loss(y, y_hat)
        costs.append(cost)
        b_hat_d, W_hat_d = get_grads(X, y, y_hat)
        b_hat, W_hat = update_params(W_hat, b_hat, W_hat_d, b_hat_d, alpha)
        if (cost > costs[-1]) or (np.round(cost, 5) <= 0.0001):
            break

    return W_hat, b_hat, y_hat, costs
weights, bias, preds, loss = fit(X,y)